# 전부 암기하지 말고 pipe line을 만들어 놓자

### 1. df를 출력하자
### 2. df에서 null 값을 처리하자
### 3. df에서 drop할거 drop 하자
### 4. df에서 노말라이즈나 인코딩 할거 인코딩 하자



In [1]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np

df = pd.read_csv('/Users/unsi/Downloads/titanic.csv')


def NullHandler(df: pd.DataFrame):
    df['Age'] = df['Age'].fillna(df['Age'].mean())
    df['Cabin'] = df['Cabin'].fillna('N')
    df['Embarked'].fillna('N')
    df['Fare'] = df['Fare'].fillna(0)
    return df


def Dropfeatures(df: pd.DataFrame):
    df = df.drop(['PassengerId', 'Name', 'Ticket'], axis=1)
    return df


def Encoder(df: pd.DataFrame):
    df['Cabin'] = df['Cabin'].str[:1]
    features = ['Cabin', 'Sex', 'Embarked']
    for feature in features:
        le = LabelEncoder()
        df[feature] = le.fit_transform(df[feature])
    return df


def DataHandler(df: pd.DataFrame):
    df = NullHandler(df)
    df = Dropfeatures(df)
    df = Encoder(df)
    return df


df = DataHandler(df)



In [2]:
label = df['Survived']
feature = df.drop('Survived', axis=1)

In [3]:
df
assert df.isnull().sum().sum() == 0
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(feature, label)

In [4]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

dt_clf = DecisionTreeClassifier(random_state=11)
rf_clf = RandomForestClassifier(random_state=11)
lr_clf = LogisticRegression(solver='liblinear')

dt_clf.fit(X_train, Y_train)
dt_pred = dt_clf.predict(X_test)
print('DecisionTreeClassifier 정확도: {0:.4f}'.format(accuracy_score(Y_test, dt_pred)))

# RandomForestClassifier 학습/예측/평가
rf_clf.fit(X_train, Y_train)
rf_pred = rf_clf.predict(X_test)
print('RandomForestClassifier 정확도:{0:.4f}'.format(accuracy_score(Y_test, rf_pred)))

# LogisticRegression 학습/예측/평가
lr_clf.fit(X_train, Y_train)
lr_pred = lr_clf.predict(X_test)
print('LogisticRegression 정확도: {0:.4f}'.format(accuracy_score(Y_test, lr_pred)))


DecisionTreeClassifier 정확도: 0.7892
RandomForestClassifier 정확도:0.8251
LogisticRegression 정확도: 0.8072


### 지금부터는 하이퍼파라미터 튜닝, 그리고 cross validation을 해보겠습니다.

In [5]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

# 하이퍼파리미터 튜닝
model = DecisionTreeClassifier()
param = dict(max_depth=[3, 4, 5], min_samples_split=[2, 3])
gscv = GridSearchCV(estimator=model, param_grid=param, scoring='accuracy', cv=3)

In [6]:
result = gscv.fit(X_train, Y_train)

In [7]:
result

GridSearchCV(cv=3, estimator=DecisionTreeClassifier(),
             param_grid={'max_depth': [3, 4, 5], 'min_samples_split': [2, 3]},
             scoring='accuracy')

In [8]:
gscv_df = pd.DataFrame(result.cv_results_)

In [9]:
best_model = gscv.best_estimator_

In [10]:
from sklearn.metrics import accuracy_score

print(accuracy_score(Y_test, best_model.predict(X_test)))

0.820627802690583


### 지금부터는 cross validation을 해보겠습니다. 라고 하려고 했지만 ㅈ도 쓸대 없어보여서 s_kfold 방식을 사용하려 합니다.

In [11]:
from sklearn.model_selection import StratifiedKFold
#그런데 그거 아시나요? train test split에서도 Stratified하게 나눈다는 사실을?
# 거기다가 gridsearch cv에서 param으로 cv를 3을 줬는데 그게 kfold니까 사실 우리는 s_kflod를 이미 해본거ㅔㅇ요

In [12]:
feature

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,3,1,22.000000,1,0,7.2500,7,2
1,1,0,38.000000,1,0,71.2833,2,0
2,3,0,26.000000,0,0,7.9250,7,2
3,1,0,35.000000,1,0,53.1000,2,2
4,3,1,35.000000,0,0,8.0500,7,2
...,...,...,...,...,...,...,...,...
886,2,1,27.000000,0,0,13.0000,7,2
887,1,0,19.000000,0,0,30.0000,1,2
888,3,0,29.699118,1,2,23.4500,7,2
889,1,1,26.000000,0,0,30.0000,2,0


In [13]:
label

0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    0
889    1
890    0
Name: Survived, Length: 891, dtype: int64

In [14]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold

stack = []
skfold = StratifiedKFold(n_splits=3)

for train_idx, test_idx in skfold.split(feature, label):
    model = DecisionTreeClassifier()
    x_train, y_train = feature.values[train_idx], label.values[train_idx]
    x_test, y_test = feature.values[test_idx], label.values[test_idx]
    model.fit(x_train, y_train)
    pred = model.predict(x_test)
    stack.append((accuracy_score(y_test, pred), model, x_test, y_test))


for idx, (acc, model, x_test, y_test) in enumerate(stack):
    print(f"Fold {idx + 1} 정확도: {acc}")
    re_evaluated_acc = accuracy_score(y_test, model.predict(x_test))
    print(f"다시 평가한 정확도: {re_evaluated_acc}")
print('for문 밖에서 model을 선언하면 인스턴스 공유해서 ㅈ댐')


Fold 1 정확도: 0.7744107744107744
다시 평가한 정확도: 0.7744107744107744
Fold 2 정확도: 0.7912457912457912
다시 평가한 정확도: 0.7912457912457912
Fold 3 정확도: 0.7912457912457912
다시 평가한 정확도: 0.7912457912457912
for문 밖에서 model을 선언하면 인스턴스 공유해서 ㅈ댐
